![DLI Header](images/DLI_Header.png)

# Text Classification #

## Sentimental Analysis ##

In this notebook, you will learn to fine-tune a pre-trained model. Specifically, we will use a model for sentiment analysis. 

**Sentiment Analysis** is the task of detecting the sentiment in text. We model this problem as a simple form of a text classification problem. For example `Gollum's performance is incredible!` has a positive sentiment while `It's neither as romantic nor as thrilling as it should be.` has a negative sentiment. In such an analysis, we need to look at sentences, and we only have two classes: "positive" and "negative". Each sentence in the training set must be labeled as one or the other. Sentiment analysis is widely used by businesses to identify customer sentiment toward products, brands, or services in online conversations and feedback.

**Table of Contents**<br>
This notebook covers the below sections: 
* Dataset
    * Download and Preprocess data
    * Labeling Data (OPTIONAL)
* Use Pre-Trained Model
    * Download Model
    * Make Predictions
* Fine-Tune a Pre-Trained Model

## Dataset ##

In this notebook, we're going to use The [Stanford Sentiment Treebank (SST-2)](https://nlp.stanford.edu/sentiment/index.html) corpus for sentiment analysis. The data contains a collection of sentences with binary labels for positive and negative. 

For text classification, NeMo requires the data to be in a specific format. Data needs to be in TAB separated files (.tsv) with two columns of sentence and label. Each line of the data file contains text sequences, where words are separated with spaces and label separated with [TAB], i.e.: `[WORD] [SPACE] [WORD] [SPACE] [WORD] [TAB] [LABEL]`

For example: 
* 
```
hide new secretions from the parental units[TAB]0
that loves its characters and communicates something rather beautiful about human nature[TAB]1
...
```

### Download and Preprocess Data ###

We have prepared the SST-2 dataset for you. It should contain three files of train.tsv, dev.tsv, and test.tsv which can be used for `training`, `validation`, and `test` respectively.

In [ ]:
import os
import wget

# set data path
DATA_DIR='data'
DATA_DIR=os.path.join(DATA_DIR, 'SST-2')

In [ ]:
# check that data folder should contain train.tsv, dev.tsv, test.tsv
!ls -l {DATA_DIR}

In [ ]:
# preview data 
print('Train:')
!head -n 5 {DATA_DIR}/train.tsv

print('Dev:')
!head -n 5 {DATA_DIR}/dev.tsv

print('Test:')
!head -n 5 {DATA_DIR}/test.tsv

The format of `train.tsv` and `dev.tsv` is close to NeMo's format except to have an extra header line at the beginning of the files. We would remove these extra lines. But `test.tsv` has different format and labels are missing for this part of the data.

In [ ]:
!sed 1d {DATA_DIR}/train.tsv > {DATA_DIR}/train_nemo_format.tsv
!sed 1d {DATA_DIR}/dev.tsv > {DATA_DIR}/dev_nemo_format.tsv

## Fine-Tune a Pre-Trained Model ##

A text classification model is typically comprised of a pre-trained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model followed by a text classification layer. For training, we can use a configuration file to define the model. The configuration (config) file consists of several important sections, including: 
* **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information
* **trainer**: Any argument to be passed to PyTorch Lightning

_Note:_ NeMo provides a template for creating the configuration file, which is recommended as a starting point, but you can create your own as long as it follows the required format. 

### Configuration File ###

In [ ]:
# define config path
MODEL_CONFIG="text_classification_config.yaml"
WORK_DIR='WORK_DIR'
os.makedirs(WORK_DIR, exist_ok=True)

In [ ]:
# download the model's configuration file 
BRANCH='main'
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

The config file for text classification, `text_classification_config.yaml`, specifies model, training, and experiment management details, such as file locations, pre-trained models, and hyperparameters. The YAML config file we downloaded provides default values for most of the parameters, but there are a few items that must be specified for this experiment.

Each YAML section is a bit easier to view using the `omegaconf` package, which allows you to access and manipulate the configuration keys using a "dot" notation. We'll take a look at the details of each section using the `OmegaConf` tool. 

In [ ]:
from omegaconf import OmegaConf

CONFIG_DIR = "/dli/task/WORK_DIR/configs"
CONFIG_FILE = "text_classification_config.yaml"

config=OmegaConf.load(CONFIG_DIR + "/" + CONFIG_FILE)

# print the entire configuration file
print(OmegaConf.to_yaml(config))

Notice that some config lines, including `model.dataset.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user. Details about the model arguments can be found in the [documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/text_classification.html#training-the-text-classification-model). 

We first need to set the num_classes in the config file which specifies the number of classes in the dataset. For SST-2, we have just two classes (0-positive and 1-negative). So we set the num_classes to 2. The model supports more than 2 classes too.

We need to specify and set the `model.train_ds.file_name`, `model.validation_ds.file_name`, and `model.test_ds.file_name` in the config file to the paths of the train, validation, and test files if they exist. 

In [ ]:
!ls $DATA_DIR

In [ ]:
# set num_classes to 2
config.model.dataset.num_classes=2

# set file paths
config.model.train_ds.file_path = os.path.join(DATA_DIR, 'train_nemo_format.tsv')
config.model.validation_ds.file_path = os.path.join(DATA_DIR, 'dev_nemo_format.tsv')

# You may change other params like batch size or the number of samples to be considered (-1 means all the samples)

# print the model section
print(OmegaConf.to_yaml(config.model))

In [ ]:
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# lets modify some trainer configs

# setup max number of steps to reduce training time for demonstration purposes of this tutorial
# Training stops when max_step or max_epochs is reached (earliest)
config.trainer.max_epochs = 1

# print the trainer section
print(OmegaConf.to_yaml(config.trainer))

Note: `OmegaConf.to_yaml()` is used to create a proper format for printing the config. Once the `text_classification_config.yaml` file has been loaded into memory, changing the configuration file will require the config variable to be re-defined.

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders will be prepared for training and evaluation. Also, the pretrained BERT model will be downloaded, which can take up to a few minutes depending on the size of the chosen BERT model.

### Download Pre-Trained Model ###

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model to another model. The default model is `bert-base-uncased`. 

In [ ]:
from nemo.collections import nlp as nemo_nlp

# complete list of supported BERT-like models
for model in nemo_nlp.modules.get_pretrained_lm_models_list(): 
    print(model)

In [ ]:
# specify the BERT-like model, you want to use
# set the `model.language_modelpretrained_model_name' parameter in the config to the model you want to use
config.model.language_model.pretrained_model_name = "bert-base-uncased"

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders will also be prepared for the training and validation.

Also, the pretrained BERT model will be automatically downloaded. Note it can take up to a few minutes depending on the size of the chosen BERT model for the first time you create the model. If your dataset is large, it also may take some time to read and process all the datasets.

In [ ]:
from nemo.collections.nlp.models import TextClassificationModel
import pytorch_lightning as pl

trainer=pl.Trainer(**config.trainer)
text_classification_model=TextClassificationModel(cfg=config.model, trainer=trainer)

### Model Training ###

In [ ]:
# start model training
trainer.fit(text_classification_model)

### Evaluate Predictions ###

for inference, we can use `trainer.test()` or `model.classifytext()`. additional [documentation](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/nlp/models/text_classification/text_classification_model.py). 

In [ ]:
eval_config = OmegaConf.create({'file_path': config.model.validation_ds.file_path, 'batch_size': 64, 'shuffle': False, 'num_samples': -1})
text_classification_model.setup_test_data(test_data_config=eval_config)
trainer.test(model=text_classification_model, verbose=False)

### Inference ###

In [ ]:
# define the list of queries for inference
queries = ['by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .', 
           'director rob marshall went out gunning to make a great one .', 
           'uneasy mishmash of styles and genres .']
           
# max_seq_length=512 is the maximum length BERT supports.       
results = text_classification_model.classifytext(queries=queries, batch_size=3, max_seq_length=512)

print('The prediction results of some sample queries with the trained model:')
for query, result in zip(queries, results):
    print(f'Query : {query}')
    print(f'Predicted label: {result}')

In [ ]:
# restart the kernel
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** 

![DLI Header](images/DLI_Header.png)